In [ ]:
!gsutil cp gs://chem_dsrc/Credit_card_overdue_prediction/dataset.zip ./


Copying gs://chem_dsrc/Credit_card_overdue_prediction/dataset.zip...
/ [1 files][704.6 KiB/704.6 KiB]                                                
Operation completed over 1 objects/704.6 KiB.                                    


In [ ]:
!unzip dataset.zip

Archive:  dataset.zip
  inflating: dataset/sample_submission.csv  
  inflating: dataset/test.csv        
  inflating: dataset/train.csv       


# Library

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
!pip install pycaret
from pycaret.classification import *
from sklearn.metrics import log_loss

from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random

     |████████████████████████████████| 266kB 16.3MB/s 
     |████████████████████████████████| 2.0MB 39.0MB/s 
     |████████████████████████████████| 1.7MB 40.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 215kB 41.1MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 14.2MB 281kB/s 
     |████████████████████████████████| 245kB 45.2MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 276kB 45.7MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 6.8MB 24.6MB/s 
     |████████████████████████████████| 1.2MB 45.3MB/s 
     |████████████████████████████████| 1.1MB 36.8MB/s 
     |████████████████████████████████| 378kB 45.2MB/s 
     |████████████████████████████████| 61kB 7.2MB/

# Data Load & Preprocessing
- 훈련에 필요없는 index 컬럼 삭제.
- missing value를 모두 NAN 문자열로 대체
- dtype object 인 컬럼들을 onehot encoding

In [ ]:
train = pd.read_csv('dataset/train.csv')
train

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0
26453,26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,1,0,0,0,NaN,2.0,-47.0,2.0
26454,26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0
26455,26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,1,0,0,0,Laborers,1.0,-59.0,2.0


In [ ]:
train = pd.read_csv('dataset/train.csv')
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 


test = pd.read_csv('dataset/test.csv')
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

submit = pd.read_csv('dataset/sample_submission.csv')

In [ ]:
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NAN,2.0,-6.0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0
26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,1,0,0,0,NAN,2.0,-47.0,2.0
26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0
26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,1,0,0,0,Laborers,1.0,-59.0,2.0


In [ ]:
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NAN,2.0,-6.0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0
26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,1,0,0,0,NAN,2.0,-47.0,2.0
26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0
26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,1,0,0,0,Laborers,1.0,-59.0,2.0


In [ ]:
object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

In [ ]:
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

In [ ]:
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

In [ ]:
clf = setup(train, target = 'credit', train_size = 0.8)

,Description,Value
0,session_id,2231
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(26457, 57)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,51
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
#logloss 적용 및 predict_proba로 제출하기 위해 metric 추가
add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7f89bf0b04d0>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [ ]:
#svm, ridge는 predict_proba 미지원으로 제외
best5 = compare_models(fold = 5, sort = 'logloss', n_select = 5, exclude=['svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.6981,0.6911,0.4236,0.6872,0.6207,0.2355,0.3191,0.7725,0.812
gbc,Gradient Boosting Classifier,0.6949,0.6507,0.4118,0.6574,0.6102,0.2165,0.3100,0.7948,8.050
lda,Linear Discriminant Analysis,0.6437,0.6073,0.3378,0.5447,0.5105,0.0129,0.0525,0.8614,0.236
nb,Naive Bayes,0.6438,0.6084,0.3356,0.5543,0.5061,0.0073,0.0488,0.8621,0.084
lr,Logistic Regression,0.6421,0.5884,0.3333,0.4124,0.5022,0.0000,0.0000,0.8694,1.030
rf,Random Forest Classifier,0.6919,0.7296,0.5197,0.6664,0.6723,0.3465,0.3541,1.0297,3.014
ada,Ada Boost Classifier,0.6928,0.6254,0.4059,0.6230,0.6039,0.2033,0.3051,1.0809,0.852
et,Extra Trees Classifier,0.6640,0.6941,0.5021,0.6406,0.6478,0.2981,0.3026,2.6115,3.210
knn,K Neighbors Classifier,0.6092,0.6430,0.4460,0.5830,0.5926,0.1890,0.1917,4.0688,0.656
dt,Decision Tree Classifier,0.6007,0.6315,0.4837,0.6105,0.6052,0.2436,0.2439,13.3847,0.240


In [18]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'logloss')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.6943,0.6615,0.4069,0.6299,0.6054,0.2051,0.3116,0.8144
1,0.6931,0.6569,0.4055,0.6291,0.6035,0.2014,0.3084,0.8150
2,0.6931,0.6460,0.4059,0.6204,0.6042,0.2052,0.3060,0.8176
3,0.6908,0.6473,0.4025,0.6291,0.5998,0.1925,0.3009,0.8195
4,0.6976,0.6652,0.4117,0.6290,0.6107,0.2186,0.3222,0.8132
Mean,0.6938,0.6554,0.4065,0.6275,0.6047,0.2045,0.3098,0.8159
SD,0.0022,0.0076,0.0030,0.0036,0.0035,0.0084,0.0071,0.0023


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6835,0,0.3987,0.6059,0.5893,0.1829,0.2852,0


In [19]:
#lda 와 nb는 losloss가 좋지만 accuracy, auc가 상대적으로 좋지 않아 커스텀모델 생성
custom_model = compare_models(fold = 5, sort = 'logloss', n_select = 6, include=['lightgbm','gbc','rf','ada', 'lda', 'nb'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.6981,0.6911,0.4236,0.6872,0.6207,0.2355,0.3191,0.7725,0.734
gbc,Gradient Boosting Classifier,0.6949,0.6507,0.4118,0.6574,0.6102,0.2165,0.3100,0.7948,8.074
lda,Linear Discriminant Analysis,0.6437,0.6073,0.3378,0.5447,0.5105,0.0129,0.0525,0.8614,0.228
nb,Naive Bayes,0.6438,0.6084,0.3356,0.5543,0.5061,0.0073,0.0488,0.8621,0.080
rf,Random Forest Classifier,0.6919,0.7296,0.5197,0.6664,0.6723,0.3465,0.3541,1.0297,2.992
ada,Ada Boost Classifier,0.6928,0.6254,0.4059,0.6230,0.6039,0.2033,0.3051,1.0809,0.852


In [20]:
blended_custom = blend_models(estimator_list = custom_model, fold = 5, optimize = 'logloss')
pred_holdout_custom = predict_model(blended_custom)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.6969,0.7271,0.4134,0.6674,0.6123,0.2196,0.3172,0.7904
1,0.6971,0.7226,0.4124,0.6921,0.6111,0.2172,0.3209,0.7900
2,0.6964,0.7152,0.4129,0.7016,0.6115,0.2201,0.3155,0.7932
3,0.6936,0.7140,0.4095,0.7176,0.6070,0.2069,0.3080,0.7937
4,0.7026,0.7215,0.4202,0.6707,0.6200,0.2391,0.3362,0.7880
Mean,0.6973,0.7201,0.4137,0.6899,0.6124,0.2205,0.3196,0.7911
SD,0.0029,0.0049,0.0035,0.0189,0.0042,0.0104,0.0093,0.0021


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6884,0.7423,0.4084,0.7201,0.5994,0.2021,0.3014,0.789


In [21]:
#자동으로 상위 5개 선택한 모델
final_model = finalize_model(blended)

#Accurary, AUC, Logloss 셋다 상위 4개인 모델 
final_model_custom = finalize_model(blended_custom)

In [22]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model_custom])
prections = prep_pipe.predict_proba(test)
prections

array([[0.13969226, 0.18657197, 0.67373578],
       [0.22530049, 0.18163675, 0.59306277],
       [0.16479577, 0.18571744, 0.64948678],
       ...,
       [0.10663974, 0.14925316, 0.7441071 ],
       [0.20136388, 0.22112046, 0.57751566],
       [0.15298671, 0.28644491, 0.56056838]])

In [23]:
i, j, k = [], [], []
for row in prections:
  i.append(row[0])
  j.append(row[1])
  k.append(row[2])

len(i), len(j), len(k)

(10000, 10000, 10000)

In [26]:
submit = pd.read_csv('/content/dataset/sample_submission.csv')

In [27]:
submit

,index,0,1,2
0,26457,0,0,0
1,26458,0,0,0
2,26459,0,0,0
3,26460,0,0,0
4,26461,0,0,0
...,...,...,...,...
9995,36452,0,0,0
9996,36453,0,0,0
9997,36454,0,0,0
9998,36455,0,0,0


In [28]:
submit['0'] = i
submit['1'] = j
submit['2'] = k
submit.head(10)

,index,0,1,2
0,26457,0.139692,0.186572,0.673736
1,26458,0.225300,0.181637,0.593063
2,26459,0.164796,0.185717,0.649487
3,26460,0.131275,0.166489,0.702236
4,26461,0.147080,0.244287,0.608632
5,26462,0.145091,0.248009,0.606899
6,26463,0.362883,0.367763,0.269355
7,26464,0.144462,0.230211,0.625326
8,26465,0.124137,0.243476,0.632387
9,26466,0.127702,0.241780,0.630518


In [30]:
submit.to_csv("1_submisison.csv",index=False)